# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
# DEPLOYMENT_URL="http://localhost:2024/"
DEPLOYMENT_URL="http://localhost:8000/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data
 
 
def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)
 
        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    if msg_type == "ai":
                        return True
    return False

In [3]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': 'be55240d-e282-4103-9ad0-ad346da1bf42', 'created_at': '2025-08-14T00:14:09.214825+00:00', 'updated_at': '2025-08-14T00:14:09.214825+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None, 'interrupts': {}, 'error': None}
be55240d-e282-4103-9ad0-ad346da1bf42


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [4]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [5]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [6]:
input = { "messages": "한국 반도체 현황. 모두 한글로 만들어줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    list_chunk.append(chunk)
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])
    # elif filter_silde_data(chunk):
    #     print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
    #     print(chunk)


##########  AI 메시지  ##########
한국 반도체 현황을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 한국 반도체 산업 역사 - 한국 반도체 산업의 기원을 탐구하고, 1960년대 시작된 반도체 조립 생산부터 현재에 이르기까지의 발전 과정을 분석합니다. 
3. 한국 반도체 산업의 세계 시장 점유율 분석 - 한국 반도체 산업이 세계 시장에서 차지하는 비중을 분석하고, 주요 국가와의 비교를 통해 글로벌 시장에서의 위치를 평가합니다. 
4. 한국 주요 반도체 제조 기업 순위 및 특성 - 한국 내 주요 반도체 제조 기업의 현재 순위와 각각의 기술적 특성 및 장점을 살펴봅니다. 
5. 한국 반도체 기술 개발 동향 - 한국의 반도체 기술 발전 현황과 주요 기술 개발 동향을 파악하며, AI 및 차세대 반도체 기술의 성장 가능성을 논의합니다. 
6. 한국 반도체 산업의 미래와 전망 - 한국 반도체 산업의 미래 성장 방향 및 세계 시장에서의 경쟁력 강화를 위한 전략적 방안을 전망합니다. 

##########  AI 메시지  ##########

    슬라이드 생성에 필요한 조사를 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드에 사용할 이미지를 검색하고 있습니다.
    
##########  AI 메시지  ##########

    슬라이드 디자인을 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드를 생성하고 있습니다.
    
##########  AI 메시지  ##########
슬라이드 생성이 완료되었습니다.


In [7]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [8]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

##########  AI 메시지  ##########
한국 반도체 현황을 주제로 슬라이드를 생성하도록 하겠습니다.
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 한국 반도체 산업 역사 - 한국 반도체 산업의 기원을 탐구하고, 1960년대 시작된 반도체 조립 생산부터 현재에 이르기까지의 발전 과정을 분석합니다. 
3. 한국 반도체 산업의 세계 시장 점유율 분석 - 한국 반도체 산업이 세계 시장에서 차지하는 비중을 분석하고, 주요 국가와의 비교를 통해 글로벌 시장에서의 위치를 평가합니다. 
4. 한국 주요 반도체 제조 기업 순위 및 특성 - 한국 내 주요 반도체 제조 기업의 현재 순위와 각각의 기술적 특성 및 장점을 살펴봅니다. 
5. 한국 반도체 기술 개발 동향 - 한국의 반도체 기술 발전 현황과 주요 기술 개발 동향을 파악하며, AI 및 차세대 반도체 기술의 성장 가능성을 논의합니다. 
6. 한국 반도체 산업의 미래와 전망 - 한국 반도체 산업의 미래 성장 방향 및 세계 시장에서의 경쟁력 강화를 위한 전략적 방안을 전망합니다. 

##########  AI 메시지  ##########

    슬라이드 생성에 필요한 조사를 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드에 사용할 이미지를 검색하고 있습니다.
    
##########  AI 메시지  ##########

    슬라이드 디자인을 시작합니다.
    
##########  AI 메시지  ##########

    슬라이드를 생성하고 있습니다.
    
##########  AI 메시지  ##########
슬라이드 생성이 완료되었습니다.


In [9]:
len(state["values"]["slides"])

7

In [10]:
state["values"]["slides"]

[{'description': '한국 반도체 현황에 대한 프레젠테이션 타이틀 페이지',
  'requirements': ['타이틀 텍스트', '부제목 또는 설명'],
  'research': True,
  'slide_type': 'title',
  'topic': '한국 반도체 현황',
  'idx': 0,
  'name': '타이틀',
  'content': '## 타이틀: 한국 반도체 현황\n\n### 개요\n\n대한민국은 세계 반도체 시장에서 중요한 위치를 차지하고 있으며, 특히 메모리 반도체 부문에서 독보적인 경쟁력을 갖추고 있습니다. 삼성전자와 SK하이닉스는 글로벌 DRAM 시장에서 73% 이상의 점유율을 보유하고 있으며, 이는 한국의 전반적 반도체 산업 성장을 이끄는 주요 원동력입니다. [1]\n\n### 시장 점유율 및 경제적 영향\n\n2022년 기준, 한국은 글로벌 반도체 시장에서 17.7%의 점유율을 차지하며, 미국에 이어 2위를 유지하고 있습니다. 이는 지난 10년간 꾸준히 지속되어 온 위치입니다. [2] 또한, 반도체는 한국 전체 수출의 18.9%를 차지하며, 이는 2022년 총 129.2억 달러에 달하는 수출액으로, 한국 경제에서 차지하는 비중이 상당히 큽니다. [3]\n\n### 산업 투자 및 정부 정책\n\n한국 정부는 반도체 산업의 글로벌 경쟁력을 유지하기 위해 산업 지원 정책을 활발히 추진하고 있습니다. 정부는 반도체 클러스터 확장과 같은 기반 시설 개발 비용을 줄이고, 민간 연구 개발에 대한 세제 혜택을 확대하는 등의 조치를 취하고 있습니다. 최근 발표된 33조 원 상당의 지원 패키지를 통해 이러한 노력을 뒷받침하고 있습니다. [4][5]\n\n### 미래 전망\n\n전망에 따르면, 한국은 2032년까지 대만을 제치고 세계에서 두 번째로 큰 반도체 생산국으로 부상할 것으로 예상됩니다. 이는 AI 및 비메모리 반도체 분야로의 확대를 통해 가능할 것입니다. 반도체 산업의 구조적 변화와 시장 수요의 지속적인 증가가 이러한 전망을 뒷

In [11]:
for s in state["values"]["slides"]:
    with open(f"{s['idx']}.html", "w", encoding="utf-8") as f:
        f.write(s['html'])